Let us demonstrate this with an example using the same dataset as we used with the boolean masks.

In [ ]:
import yt
ds = yt.load("Enzo_64/DD0042/data0042")

The only argument to a cut region is a conditional on field output from a data object.  The only catch is that you *must* denote the data object in the conditional as "obj" regardless of the actual object's name.  

Here we create three new data objects which are copies of the all_data object (a region object covering the entire spatial domain of the simulation), but we've filtered on just "hot" material, the "dense" material, and the "overpressure and fast" material.

In [ ]:
ad = ds.all_data()
hot_ad = ad.cut_region(['obj["gas", "temperature"] > 1e6'])
dense_ad = ad.cut_region(['obj["gas", "density"] > 5e-30'])

# you can chain cut regions in two ways:
dense_and_cool_ad = dense_ad.cut_region(['obj["gas", "temperature"] < 1e5'])
overpressure_and_fast_ad = ad.cut_region(['(obj["gas", "pressure"] > 1e-14) & (obj["gas", "velocity_magnitude"].in_units("km/s") > 1e2)'])

You can also construct a cut_region using the include_ and exclude_ functions as well.

In [ ]:
ad = ds.all_data()
hot_ad = ad.include_above(("gas", "temperature"), 1e6)
dense_ad = ad.include_above(("gas", "density"), 5e-30)

# These can be chained as well
dense_and_cool_ad = dense_ad.include_below(("gas", "temperature"), 1e5)
overpressure_and_fast_ad = ad.include_above(("gas", "pressure"), 1e-14)
overpressure_and_fast_ad = overpressure_and_fast_ad.include_above(("gas", "velocity_magnitude"), 1e2, 'km/s')

Upon inspection of our "hot_ad" object, we can still get the same results as we got with the boolean masks example above:

In [ ]:
print ("Temperature of all cells:\n ad['temperature'] = \n%s\n" % ad["gas", "temperature"])
print ("Temperatures of all \"hot\" cells:\n hot_ad['temperature'] = \n%s" % hot_ad["gas", "temperature"])

In [ ]:
print ("Density of dense, cool material:\n dense_and_cool_ad['density'] = \n%s\n" % dense_and_cool_ad["gas", "density"])
print ("Temperature of dense, cool material:\n dense_and_cool_ad['temperature'] = \n%s" % dense_and_cool_ad["gas", "temperature"])

Now that we've constructed a `cut_region`, we can use it as a data source for further analysis. To create a plot based on a `cut_region`, use the `data_source` keyword argument provided by yt's plotting objects.

Here's an example using projections:

In [ ]:
proj1 = yt.ProjectionPlot(ds, 'x', ("gas", "density"), weight_field=("gas", "density"))
proj1.annotate_title('No Cuts')
proj1.set_figure_size(5)
proj1.show()

proj2 = yt.ProjectionPlot(ds, 'x', ("gas", "density"), weight_field=("gas", "density"), data_source=hot_ad)
proj2.annotate_title('Hot Gas')
proj2.set_zlim(("gas", "density"), 3e-31, 3e-27)
proj2.set_figure_size(5)
proj2.show()

The `data_source` keyword argument is also accepted by `SlicePlot`, `ProfilePlot` and `PhasePlot`:

In [ ]:
slc1 = yt.SlicePlot(ds, 'x', ("gas", "density"), center='m')
slc1.set_zlim(("gas", 'density'), 3e-31, 3e-27)
slc1.annotate_title('No Cuts')
slc1.set_figure_size(5)
slc1.show()

slc2 = yt.SlicePlot(ds, 'x', ("gas", "density"), center='m', data_source=dense_ad)
slc2.set_zlim(("gas", 'density'), 3e-31, 3e-27)
slc2.annotate_title('Dense Gas')
slc2.set_figure_size(5)
slc2.show()

In [ ]:
ph1 = yt.PhasePlot(ad, ("gas", "density"), ("gas", "temperature"), ("gas", "mass"), weight_field=None)
ph1.set_xlim(3e-31, 3e-27)
ph1.annotate_title('No Cuts')
ph1.set_figure_size(5)
ph1.show()

ph1 = yt.PhasePlot(dense_ad, ("gas", "density"), ("gas", "temperature"), ("gas", "mass"), weight_field=None)
ph1.set_xlim(3e-31, 3e-27)
ph1.annotate_title('Dense Gas')
ph1.set_figure_size(5)
ph1.show()